In [1]:
import numpy as np
import pandas as pd

# Read the adjacency matrix from an Excel file
def read_graph_from_excel(file_path):
    df = pd.read_excel(file_path, index_col=0)
    graph = df.to_numpy()
    graph[graph == np.inf] = float('inf')  # Handle infinite values correctly
    return graph

class TSPSolver:
    def __init__(self, graph):
        self.graph = graph
        self.n = len(graph)
        self.final_res = float('inf')
        self.final_path = []

    def copy_to_final(self, curr_path):
        self.final_path = curr_path[:] + [curr_path[0]]

    def first_min(self, i):
        min_cost = float('inf')
        for k in range(self.n):
            if self.graph[i][k] < min_cost and i != k:
                min_cost = self.graph[i][k]
        return min_cost

    def second_min(self, i):
        first, second = float('inf'), float('inf')
        for j in range(self.n):
            if i == j:
                continue
            if self.graph[i][j] <= first:
                second = first
                first = self.graph[i][j]
            elif self.graph[i][j] <= second and self.graph[i][j] != first:
                second = self.graph[i][j]
        return second

    def TSP_rec(self, curr_bound, curr_weight, level, curr_path):
        if level == self.n:
            if self.graph[curr_path[level - 1]][curr_path[0]] != float('inf'):
                curr_res = curr_weight + self.graph[curr_path[level - 1]][curr_path[0]]
                if curr_res < self.final_res:
                    self.copy_to_final(curr_path)
                    self.final_res = curr_res
            return

        for i in range(self.n):
            if self.graph[curr_path[level-1]][i] != float('inf') and not self.visited[i]:
                temp = curr_bound
                curr_weight += self.graph[curr_path[level - 1]][i]

                if level == 1:
                    curr_bound -= ((self.first_min(curr_path[level - 1]) + self.first_min(i)) / 2)
                else:
                    curr_bound -= ((self.second_min(curr_path[level - 1]) + self.first_min(i)) / 2)

                if curr_bound + curr_weight < self.final_res:
                    curr_path[level] = i
                    self.visited[i] = True
                    self.TSP_rec(curr_bound, curr_weight, level + 1, curr_path)

                curr_weight -= self.graph[curr_path[level - 1]][i]
                curr_bound = temp
                self.visited = [False] * self.n
                for j in range(level):
                    if curr_path[j] != -1:
                        self.visited[curr_path[j]] = True

    def TSP(self):
        curr_bound = 0
        curr_path = [-1] * self.n
        self.visited = [False] * self.n

        for i in range(self.n):
            curr_bound += (self.first_min(i) + self.second_min(i))

        curr_bound = np.ceil(curr_bound / 2)
        self.visited[0] = True
        curr_path[0] = 0

        self.TSP_rec(curr_bound, 0, 1, curr_path)
        return self.final_res, self.final_path


file_path = 'C:\\Users\\Admin\\Documents\\Zalo Received Files\\Results.xls'  # Path to your Excel file
graph = read_graph_from_excel(file_path)

solver = TSPSolver(graph)
optimal_distance, optimal_path = solver.TSP()
print("Optimal distance:", optimal_distance)
print("Optimal path:", optimal_path)


Optimal distance: 5834.450999999999
Optimal path: [0, 6, 5, 1, 2, 3, 4, 11, 10, 8, 7, 9, 0]
